# 1. Specify all parameters to be varied


In [1]:
from fedt import *

In [2]:
experiment = FEDTExperiment()

experiment.CAD_variables = [{
        'name' : 'side length',
        'data type' : 'ratio',
        'test_values' : [x for x in range(10, 20, 5)],
        'argname' : 'side length'
    }]
experiment.CAM_variables = [{
        'name' : 'layer height',
        'data type' : 'ratio',
        'test_values' : [x / 10.0 for x in range(5, 20, 5)], # i hate this so much, but we would need numpy arange to make it better
        'argname' : '--layer-height'
    },
    {
        'name' : 'fill pattern',
        'data type' : 'nominal',
        'test_values' : ['triangles', 'concentric'], # I would like them all to have the same named thing so I can call it
        'argname' : '--fill-pattern'
    }]

experiment.fab_repetitions = 2

sample_post_process_variable = {
    'name' : 'paint type',
    'data type' : 'nominal',
    'test_values' : ['lacquer', 'spraypaint'],
    'description' : 'use the {} on the object'
}
experiment.post_process_variables = []
experiment.post_process_repetitions = 1

sample_interaction_variable = {
    'name' : 'user push force level',
    'data type' : 'ratio',
    'test_values' : [x for x in range(10)],
    'instruction' : 'push on the button with force {} / 10' 
}
experiment.interaction_variables = [sample_interaction_variable]

experiment.measurement_variables = [{
        'name' : 'printed x-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed y-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed z-axis',
        'data type' : 'ratio'
    }
]
experiment.measurement_repetitions = 5

tea_hypothesis = (['material', 'measured z-axis' ], ['fill_pattern:concentric > triangle'])

###########

experiment.experiment_size()
experiment.label_all_conditions()

0

This experiment will require fabricating 24 unique objects.
Users will perform 600 interactions.
360 total measurements will be recorded.


{((('CAD', 'side length', 10),
   ('CAM', '--layer-height', 0.5),
   ('CAM', '--fill-pattern', 'triangles')),): 'L12',
 ((('CAD', 'side length', 10),
   ('CAM', '--layer-height', 0.5),
   ('CAM', '--fill-pattern', 'concentric')),): 'L13',
 ((('CAD', 'side length', 10),
   ('CAM', '--layer-height', 1.0),
   ('CAM', '--fill-pattern', 'triangles')),): 'L14',
 ((('CAD', 'side length', 10),
   ('CAM', '--layer-height', 1.0),
   ('CAM', '--fill-pattern', 'concentric')),): 'L15',
 ((('CAD', 'side length', 10),
   ('CAM', '--layer-height', 1.5),
   ('CAM', '--fill-pattern', 'triangles')),): 'L16',
 ((('CAD', 'side length', 10),
   ('CAM', '--layer-height', 1.5),
   ('CAM', '--fill-pattern', 'concentric')),): 'L17',
 ((('CAD', 'side length', 15),
   ('CAM', '--layer-height', 0.5),
   ('CAM', '--fill-pattern', 'triangles')),): 'L18',
 ((('CAD', 'side length', 15),
   ('CAM', '--layer-height', 0.5),
   ('CAM', '--fill-pattern', 'concentric')),): 'L19',
 ((('CAD', 'side length', 15),
   ('CAM', '-

# 2. CAD

In [3]:
# FreeCAD create cube    
def geometry_function(App, side_length='20 mm'):
    App.ActiveDocument.addObject("Part::Box","Box")
    App.ActiveDocument.ActiveObject.Label = "Cube"
    App.ActiveDocument.recompute()
    FreeCAD.ActiveDocument.getObject('Box').Width = side_length
    FreeCAD.ActiveDocument.getObject('Box').Length = side_length
    FreeCAD.ActiveDocument.getObject('Box').Height = side_length
    App.ActiveDocument.recompute()

def label_function(App, label_string="L0"):
    # https://wiki.freecad.org/Draft_ShapeString_tutorial
    ss=Draft.make_shapestring(String=label_string, FontFile=FONT_LOCATION, Size=5.0, Tracking=0.0)
    plm=FreeCAD.Placement()
    plm.Base=FreeCAD.Vector(4.0, 4.0, side=20)
    plm.Rotation.Q=(0.0, 0.0, 0, 1.0)
    ss.Placement=plm
    ss.Support=None
    Draft.autogroup(ss)
    FreeCAD.ActiveDocument.recompute()
    
    App.ActiveDocument.addObject('Part::Extrusion','Extrude')
    f = App.ActiveDocument.getObject('Extrude')
    f.Base = App.ActiveDocument.getObject('ShapeString')
    f.DirMode = "Normal"
    f.DirLink = None
    f.LengthFwd = 5.000000000000000
    f.LengthRev = 0.000000000000000
    f.Solid = False
    f.Reversed = False
    f.Symmetric = False
    f.TaperAngle = 0.000000000000000
    f.TaperAngleRev = 0.000000000000000
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('ShapeString').Visibility = False
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('Extrude').Placement = App.Placement(App.Vector(2,2,0),App.Rotation(App.Vector(0,0,1),0))
    App.ActiveDocument.recompute()
    #FreeCAD.ActiveDocument.getObject('Extrude').Shape.exportStl('ext.stl')

experiment.configure_for_freecad()
experiment.label_function = label_function
experiment.geometry_function = geometry_function

ModuleNotFoundError: No module named 'PySide2'

# 3. CAM

In [4]:
experiment.configure_for_printing()

# 4. Fabrication


In [5]:
experiment.fabricate()

IndexError: tuple index out of range

# 5. Post-Processing

In [6]:
experiment.post_process()

TypeError: post_process() takes 1 positional argument but 2 were given

# 6. User Interaction

In [7]:
experiment.interact()

AttributeError: 'list' object has no attribute 'values'

# 7. Measurement

In [8]:
experiment.measure()

now please fill <fedt.FEDTExperiment object at 0x107c3d550> with data


# 8. Tea-based hypothesis testing

In [9]:
#from fedt_tea import do_experiment

'''tea_results = do_experiment(measurement_variables,
                            tea_hypothesis,
                            experiment_csv,
                            CAD_variables,
                            CAM_variables,
                            post_process_variables,
                            interaction_variables)'''

NameError: name 'measurement_variables' is not defined

# 9. Reporting

In [10]:
experiment.report_latex()

AttributeError: 'FEDTExperiment' object has no attribute 'tea_results'

In [11]:
'''We used {cad_tool} to generate objects varying along the following dimensions: {CAD_variables}.
        We then used {cam_tool} to generate different CAM settings: {CAM_variables}.
        We fabricated objects of each configuration {fab_repetitions} times on our {machine}.
        In all, we fabricated {num_fabbed_objects} objects.
        After fabrication, we performed post-processing, processing {post_process_repetitions} objects with each of the following values: {post_process_variables}.
        Users then interacted with our objects, doing {interaction_variables} {measurement_repetitions} times on every object ({num_user_ixns} total interactions).
        We recorded {measurement_variables} for each ({num_recorded_values} total measurements).'''.format(
                **{
                    'cad_tool': str(experiment.cad_tool),
                    'CAD_variables': str(experiment.CAD_variables),
                    'cam_tool': str(os.path.split(experiment.cam_tool)[-1]),
                    'CAM_variables': str(experiment.CAM_variables),
                    'fab_repetitions': str(experiment.fab_repetitions),
                    'post_process_variables': str(experiment.post_process_variables),
                    'post_process_repetitions': str(experiment.post_process_repetitions),
                    'interaction_variables': str(experiment.interaction_variables),
                    'measurement_variables': str(experiment.measurement_variables),
                    'measurement_repetitions': str(experiment.measurement_repetitions),
                    'machine': str(experiment.machine),
                    'num_fabbed_objects': str(experiment.number_of_fabbed_objects),
                    'num_recorded_values': str(experiment.number_of_recorded_values),
                    'num_user_ixns': str(experiment.number_of_user_interactions),
                })

"We used None to generate objects varying along the following dimensions: [{'name': 'side length', 'data type': 'ratio', 'test_values': [10, 15], 'argname': 'side length'}].\n        We then used PrusaSlicer to generate different CAM settings: [{'name': 'layer height', 'data type': 'ratio', 'test_values': [0.5, 1.0, 1.5], 'argname': '--layer-height'}, {'name': 'fill pattern', 'data type': 'nominal', 'test_values': ['triangles', 'concentric'], 'argname': '--fill-pattern'}].\n        We fabricated objects of each configuration 2 times on our PRUSA.\n        In all, we fabricated 24 objects.\n        After fabrication, we performed post-processing, processing 1 objects with each of the following values: [].\n        Users then interacted with our objects, doing [{'name': 'user push force level', 'data type': 'ratio', 'test_values': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'instruction': 'push on the button with force {} / 10'}] 5 times on every object (600 total interactions).\n        We recorded